# The `instrument` package for Bluesky Data Acquisition

*APS Training for Bluesky Data Acquisition*.

**Objective**

In this notebook, we describe the `instrument` package for a simulated X-ray instrument at a user facility such as the Advanced Photon Source.  The goal is to use EPICS as much as possible to provide the control system features and to use Bluesky (providing the data acquisition framework) as a thin layer on top of EPICS.

**Contents**

* [Start the `instrument` package](#start-the-instrument-package)
* [Description](#Description)
  * [`adsimdet`](#adsimdet)
  * [`scaler1`](#scaler1)
  * [`temperature`](#temperature)
  * [motors](#motors)
  * [`noisy`](#noisy)
  * [plans](#plans)
* [Log files](#log-files)
* [SPEC data files](#spec-data-files)
* [User code file](#user-code-file)
* [`databroker`](#databroker)

**Overview**

This simulated instrument is provided using [docker](https://www.docker.com/) images
for [EPICS base](https://www.aps.anl.gov/epics), the [synApps](https://www.aps.anl.gov/BCDA/synApps) [xxx](https://github.com/epics-modules/xxx) module, and [EPICS area detector](https://areadetector.github.io/master/index.html) [ADSimDetector](https://areadetector.github.io/master/ADSimDetector/simDetector.html?highlight=adsimdetector).  The images are based on these software versions:

* Ubuntu 20.04
* EPICS base 7.0.3
* synApps 6.1
* area detector 3.7

Two EPICS IOCs are provided:

prefix | description | docker image | documentation
:--- | :--- | :--- | :---
`ad:` | area detector IOC | [`prjemian/synapps-6.1-ad-3.7`](https://hub.docker.com/r/prjemian/synapps-6.1-ad-3.7) | https://github.com/prjemian/epics-docker/tree/main/n4_areaDetector
`gp:` | general purpose IOC | [`prjemian/synApps-6.1`](https://hub.docker.com/r/prjemian/synapps-6.1) | https://github.com/prjemian/epics-docker/tree/main/n3_synApps

## Start the `instrument` package

In [1]:
from instrument.collection import *

I Tue-11:31:54 - ############################################################ startup
I Tue-11:31:54 - logging started
I Tue-11:31:54 - logging level = 10
I Tue-11:31:54 - /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/instrument/collection.py
I Tue-11:31:54 - /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/instrument/mpl/notebook.py


Activating auto-logging. Current session state plus future input saved.
Filename       : /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/.logs/ipython_console.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


I Tue-11:31:54 - /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/instrument/iocs/check_iocs.py
I Tue-11:31:54 - EPICS IOCs ready...
I Tue-11:31:54 - #### Bluesky Framework ####
I Tue-11:31:54 - /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/instrument/framework/check_python.py
I Tue-11:31:54 - /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/instrument/framework/check_bluesky.py
I Tue-11:31:55 - /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/instrument/framework/initialize.py
I Tue-11:31:55 - using databroker catalog 'training'
I Tue-11:31:55 - /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/instrument/framework/metadata.py
I Tue-11:31:55 - #### Devices ####
I Tue-11:31:55 - /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/instrument/devices/area_detector.py
I Tue-11:31:55 - /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/instrument/devices/calculation_records.py
I Tue-11:31:57 - /home/prjemian/Document

## Description

Might be a good idea to know now what this instrument package provides.  

Notably, the table includes:

ophyd name(s) | Description | label(s)
:--- | :---| :---
`adsimdet` | simulated EPICS area detector | `area_detector`
`calcs` & `calcouts` | calculation support |
`gp_stats` | details about the general purpose IOC |
`I0`, `diode`, ... | named scaler channels | `counter`
`m1` .. `m16` | 16 simulated EPICS motors | `motor`
`noisy` | simulated diffraction peak |
`scaler1` | simulated 16-channel EPICS scaler | `scalers`
`shutter` | simulated shutter | 
`temperature` | simulated temperature controller |

### `adsimdet`

The EPICS Area Detector ADSimDetector is included with this instrument as `adsimdet`.  The detector is a monochrome 1k x 1k frame.  The image is written to an HDF5 file and made available to the [`databroker`](#databroker) via a shared volume from the docker container.  `adsimdet` can be used as a detector.

The image is a simulated diffraction spot with center randomly-chosen between 100..900 on both axes.  The width is also random as is the peak intensity and noise level.  Furthermore, to simulate realistic conditions, the center position of both axes is adjusted by a few pixels using a pair of *swait* records (`gp:userCalc9` & `gp:userCalc10`) updating from random numbers.

In [2]:
adsimdet.summary()

data keys (* hints)
-------------------

read attrs
----------
hdf1                 MyHDF5Plugin        ('adsimdet_hdf1')

config keys
-----------
adsimdet_cam_acquire_period
adsimdet_cam_acquire_time
adsimdet_cam_image_mode
adsimdet_cam_manufacturer
adsimdet_cam_model
adsimdet_cam_num_exposures
adsimdet_cam_num_images
adsimdet_cam_trigger_mode

configuration attrs
-------------------
cam                  MyFixedCam          ('adsimdet_cam')
cam.acquire_period   EpicsSignalWithRBV  ('adsimdet_cam_acquire_period')
cam.acquire_time     EpicsSignalWithRBV  ('adsimdet_cam_acquire_time')
cam.image_mode       EpicsSignalWithRBV  ('adsimdet_cam_image_mode')
cam.manufacturer     EpicsSignalRO       ('adsimdet_cam_manufacturer')
cam.model            EpicsSignalRO       ('adsimdet_cam_model')
cam.num_exposures    EpicsSignalWithRBV  ('adsimdet_cam_num_exposures')
cam.num_images       EpicsSignalWithRBV  ('adsimdet_cam_num_images')
cam.trigger_mode     EpicsSignalWithRBV  ('adsimdet_cam_trigger_m

### `scaler1`

A simulated 16-channel scaler is configured with several channels of pulse counters, typical of a synchrotron beam line.  `scaler1` can be used as a detector.

In [3]:
scaler1.summary()

data keys (* hints)
-------------------
*I0
*I00
*diode
*roi1
*roi2
 scaler1_time
*scint
*timebase

read attrs
----------
channels             Channels            ('scaler1_channels')
channels.chan01      ScalerChannel       ('scaler1_channels_chan01')
channels.chan01.s    EpicsSignalRO       ('timebase')
channels.chan02      ScalerChannel       ('scaler1_channels_chan02')
channels.chan02.s    EpicsSignalRO       ('I0')
channels.chan03      ScalerChannel       ('scaler1_channels_chan03')
channels.chan03.s    EpicsSignalRO       ('scint')
channels.chan04      ScalerChannel       ('scaler1_channels_chan04')
channels.chan04.s    EpicsSignalRO       ('diode')
channels.chan05      ScalerChannel       ('scaler1_channels_chan05')
channels.chan05.s    EpicsSignalRO       ('I00')
channels.chan11      ScalerChannel       ('scaler1_channels_chan11')
channels.chan11.s    EpicsSignalRO       ('roi1')
channels.chan12      ScalerChannel       ('scaler1_channels_chan12')
channels.chan12.s    EpicsSign

The timebase is always shown.  The additional channels (using names from the scaler GUI screen) are shown output of `listdevice(scaler1)` below.

Note that in the instrument package, the channel names were assigned on startup.  An operating beam line would not do define the names here but instead, let the instrument team name these channels in the GUI screen at the time the cables from the pulse detectors are connected physically to the scaler.

Count the scaler from the command line:

In [4]:
%ct scalers

[This data will not be saved. Use the RunEngine to collect data.]
timebase                       11000000.0
I0                             6.0
scint                          4.0
diode                          5.0
I00                            5.0
roi1                           0.0
roi2                           0.0
scaler1_time                   1.1


NOTE: the actual counts are random numbers.  Also, the EPICS soft scaler seems to add 0.1s to the counting time.  Might be a bug with the EPICS scaler code.

In [5]:
listdevice(scaler1)

============ ========== ==========================
name         value      timestamp                 
============ ========== ==========================
timebase     11000000.0 2021-04-13 11:32:02.253280
I0           6.0        2021-04-13 11:32:02.253280
scint        4.0        2021-04-13 11:32:02.253280
diode        5.0        2021-04-13 11:32:02.253280
I00          5.0        2021-04-13 11:32:02.253280
roi1         0.0        2021-04-13 11:32:02.253280
roi2         0.0        2021-04-13 11:32:02.253280
scaler1_time 1.1        2021-04-13 11:13:19.111873
============ ========== ==========================



As an added convenience, shortcut names for these channels were also assigned to local Python symbols (since the object path to the channels is easy to forget) in `instrument/devices/scaler.py` with these lines:

```python
timebase = scaler1.channels.chan01.s
I0 = scaler1.channels.chan02.s
scint = scaler1.channels.chan03.s
diode = scaler1.channels.chan04.s
```

This command: `scaler1.select_channels()` configures `scaler` to use just the channels with non-empty names.  To limit data collection to a subset of these channels, name the channels in a list argument, such as: `scaler1.select_channels(['I0', 'diode'])`

### `temperature`

The temperature simulator, implemented in `gp:userCalc8` works like a temperature controller, with setpoint and readback values.    `temperature` can be used as a detector **and** as a positioner (like a motor).

Additional controls for noise, update interval, maximum change per update, in-position tolerance, and done are provided, either through EPICS or in Python.  Once a new setpoint is entered, the readback will progress towards it, limited by the max change and the update interval.  Once at the setpoint, noise continues to be added to the temperature to provide realism.

In [6]:
temperature.summary()
listdevice(temperature)

data keys (* hints)
-------------------
*temperature
*temperature_setpoint

read attrs
----------
setpoint             EpicsSignal         ('temperature_setpoint')
readback             EpicsSignal         ('temperature')

config keys
-----------
temperature_calculation
temperature_description
temperature_max_change
temperature_noise
temperature_previous_value_pv
temperature_scanning_rate
temperature_tolerance

configuration attrs
-------------------
calculation          EpicsSignal         ('temperature_calculation')
description          EpicsSignal         ('temperature_description')
max_change           EpicsSignal         ('temperature_max_change')
noise                EpicsSignal         ('temperature_noise')
previous_value_pv    EpicsSignal         ('temperature_previous_value_pv')
scanning_rate        EpicsSignal         ('temperature_scanning_rate')
tolerance            EpicsSignal         ('temperature_tolerance')

unused attrs
------------
done                 Signal          

### motors

There are 16 soft motor channels: `gp:m1` .. `gp:m16`.  Any motor can be used as a detector **and** as a positioner.

The first motor, `gp:m1`, is used to compute a simulated 1-D diffraction peak ([`noisy`](#noisy)) using `gp:userCalc1`.

To make it easier to change the motor step size, a custom `MyEpicsMotor` class was made, subclassing from `ophyd.EpicsMotor` and adding a component to access the motor's `.SREV` (steps/revolution) field.  The default motor step size is 0.01 (with SREV=200).  For only `m1`, the step size has been changed to 0.001 by setting SREV=2000.

NOTE: Changing SREV (in this simulator* seems to affect the soft motor's simulated hardware limit.  Be careful if you adjust SREV to higher numbers (smaller step sizes) that the expected range of motion remains sufficient for your measurements.  Again, another thing to change in the EPICS support.

In [7]:
m1.summary()
listdevice(m1)

data keys (* hints)
-------------------
*m1
 m1_user_setpoint

read attrs
----------
user_readback        EpicsSignalRO       ('m1')
user_setpoint        EpicsSignal         ('m1_user_setpoint')

config keys
-----------
m1_acceleration
m1_motor_egu
m1_user_offset
m1_user_offset_dir
m1_velocity

configuration attrs
-------------------
user_offset          EpicsSignal         ('m1_user_offset')
user_offset_dir      EpicsSignal         ('m1_user_offset_dir')
velocity             EpicsSignal         ('m1_velocity')
acceleration         EpicsSignal         ('m1_acceleration')
motor_egu            EpicsSignal         ('m1_motor_egu')

unused attrs
------------
offset_freeze_switch EpicsSignal         ('m1_offset_freeze_switch')
set_use_switch       EpicsSignal         ('m1_set_use_switch')
motor_is_moving      EpicsSignalRO       ('m1_motor_is_moving')
motor_done_move      EpicsSignalRO       ('m1_motor_done_move')
high_limit_switch    EpicsSignalRO       ('m1_high_limit_switch')
low_limit_s

### `noisy`

A simulated diffraction peak is computed in 1-D using a Lorentzian function in EPICS PV `gp:userCalc1`.  The simulation uses the position of motor `gp:m1` and random choices for center, width, noise, and peak intensity (scale factor).  `noisy` can be used as a detector.

NOTE: `noisy` is an `ophyd.EpicsSignal` (variant), thus it lacks the `.summary()` method that `ophyd.Device` objects have.

In [8]:
listdevice(noisy)

===== ================ ==========================
name  value            timestamp                 
===== ================ ==========================
noisy 38.1874849819879 2021-04-13 11:31:58.524389
===== ================ ==========================



### plans

Several examples of user plan are provided in file `instrument/plans/peak_finder_example.py`.  These plans are described and used in the notebook [Lineup a 1-D peak](https://nbviewer.jupyter.org/github/BCDA-APS/bluesky_instrument_training/blob/main/lineup_1d_peak.ipynb).

## Log files

For diagnostic and general support, log files are created to record activity.  In the working directory, the log files are written to a `./.logs` subdirectory.

There are two kinds of file, one that records user commands and the python
result, the other records items sent to the
[customized](https://github.com/prjemian/stdlogpj#example-directing-logs-to-a-specific-directory)
Python [logging](https://docs.python.org/3/library/logging.html) package.

In the IPython session, use the `!` to run a linux command:

In [9]:
!ls -lAFgh .logs

total 116K
-rw-rw-r-- 1 prjemian  707 Apr 13 11:32 ipython_console.log
-rw-rw-r-- 1 prjemian 1.5K Apr 13 11:19 ipython_console.log.001~
-rw-rw-r-- 1 prjemian 2.0K Apr 13 11:12 ipython_console.log.002~
-rw-rw-r-- 1 prjemian 4.2K Apr 13 11:08 ipython_console.log.003~
-rw-rw-r-- 1 prjemian  953 Apr 13 10:58 ipython_console.log.004~
-rw-rw-r-- 1 prjemian 1.7K Apr 13 10:54 ipython_console.log.005~
-rw-rw-r-- 1 prjemian   20 Apr 13 10:53 ipython_console.log.006~
-rw-rw-r-- 1 prjemian   20 Apr 13 10:52 ipython_console.log.007~
-rw-rw-r-- 1 prjemian   20 Apr 13 10:50 ipython_console.log.008~
-rw-rw-r-- 1 prjemian   20 Apr 13 10:49 ipython_console.log.009~
-rw-rw-r-- 1 prjemian  976 Apr 13 10:28 ipython_console.log.010~
-rw-rw-r-- 1 prjemian 2.1K Apr 12 17:40 ipython_console.log.011~
-rw-rw-r-- 1 prjemian  60K Apr 13 11:32 ipython_logger.log


Let's take a look at a few lines of each type of file, to get a feel for the information logged.

The `ipython_console.log` file is created for every session (every time IPython is started or every time the Jupyter kernel is started).  It records the commands that were entered and the output, if any, from each command.  It does not record anything that was sent to the console by `print()` statements.  The older log files are numbered, higher number is older (each new session shifts these numbers by 1).

In [10]:
!head .logs/ipython_console.log

# IPython log file

# Tue, 13 Apr 2021 11:32:00
adsimdet.summary()
# Tue, 13 Apr 2021 11:32:01
scaler1.summary()
# Tue, 13 Apr 2021 11:32:01
get_ipython().run_line_magic('ct', 'scalers')
# Tue, 13 Apr 2021 11:32:02
listdevice(scaler1)


The `ipython_logger.log` file(s) contain the output from calls to the `logger`.  The files are appended with new logger reports until the file reaches ca. 1 MB.  (Larger files are slow to append.)  Then the file is given a number and a new logger file is created.  Higher numbers are older.  At most, 9 numbered files are retained (to avoid filling disk storage with unnecessary diagnostics).

A single *logger* file may contain reports from several sessions.  The third piece of information is the `pid` (process identifier) of the session.  The PID is assigned by the operating system when the session is started.

The logger file contains a full report while the report's representation is more terse.  (Users do not always want the full details.  Just remember they are available in the logger file.)  Here's the last few logger lines as shown on the console:

    I Wed-00:24:52 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/devices/temperature_signal.py
    I Wed-00:24:52 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/plans/peak_finder_example.py
    I Wed-00:24:52 - Startup is complete.

Those same lines appear in the logger file as:

    |2021-02-24 00:24:52.091|INFO|25986|bluesky-session|temperature_signal|11|MainThread| - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/devices/temperature_signal.py
    |2021-02-24 00:24:52.165|INFO|25986|bluesky-session|peak_finder_example|15|MainThread| - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/plans/peak_finder_example.py
    |2021-02-24 00:24:52.179|INFO|25986|bluesky-session|collection|23|MainThread| - Startup is complete.


In [11]:
!tail .logs/ipython_logger.log

|2021-04-13 11:32:00.646|INFO|112203|bluesky-session|spec_data_file_writer|13|MainThread| - /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/instrument/callbacks/spec_data_file_writer.py
|2021-04-13 11:32:00.654|INFO|112203|bluesky-session|spec_data_file_writer|29|MainThread| - writing to SPEC file: /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/20210413-113200.dat
|2021-04-13 11:32:00.656|INFO|112203|bluesky-session|spec_data_file_writer|30|MainThread| -    >>>>   Using default SPEC file name   <<<<
|2021-04-13 11:32:00.657|INFO|112203|bluesky-session|spec_data_file_writer|31|MainThread| -    file will be created when bluesky ends its next scan
|2021-04-13 11:32:00.659|INFO|112203|bluesky-session|spec_data_file_writer|32|MainThread| -    to change SPEC file, use command:   newSpecFile('title')
|2021-04-13 11:32:00.660|INFO|112203|bluesky-session|collection|21|MainThread| - #### Plans ####
|2021-04-13 11:32:00.664|INFO|112203|bluesky-session|peak_finder_example|

## SPEC data files

A [SPEC](https://certif.com) data file records a copy of the scan data (area detector images are not stored in SPEC files).  By default, the file is created in the present working directory using a name constructed from the date and time with a `.dat` file extension.  Unlike SPEC, new data is written *at the end of a scan*.  It is also possible to use the [`SpecWriterCallback`](https://apstools.readthedocs.io/en/latest/source/_filewriters.html#apstools.filewriters.SpecWriterCallback) to write data extracted from the database after the experiment is done.

<details>
<summary>Example SPEC data file from a bluesky session:</summary>

```
#F /home/prjemian/Documents/projects/BCDA-APS/epics-bluesky-vm/20210412-173925.dat
#E 1618267165
#D Mon Apr 12 17:39:25 2021
#C Bluesky  user = prjemian  host = zap
#O0 
#o0 

#S 34  count(detectors=['temperature'], num=1)
#D Mon Apr 12 17:39:34 2021
#C Mon Apr 12 17:39:34 2021.  plan_type = generator
#C Mon Apr 12 17:39:34 2021.  uid = d94d2126-fd52-461c-9284-23e5f34b326b
#MD uid = d94d2126-fd52-461c-9284-23e5f34b326b
#MD beamline_id = APS EPICS/Python/Bluesky training VM
#MD detectors = ['temperature']
#MD instrument_name = training
#MD login_id = prjemian@zap
#MD notebook = watch_temperature
#MD num_intervals = 0
#MD num_points = 1
#MD pid = 59652
#MD proposal_id = training-2021-04
#MD versions = {'apstools': '1.4.1', 'bluesky': '1.6.7', 'databroker': '1.2.2', 'epics': '3.4.3', 'h5py': '3.2.1', 'intake': '0.6.2', 'matplotlib': '3.3.4', 'numpy': '1.20.1', 'ophyd': '1.6.1', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}
#P0 
#N 4
#L Epoch_float  Epoch  temperature_setpoint  temperature
0.2451648712158203 0 35.0 34.51313801785306
#C Mon Apr 12 17:39:34 2021.  num_events_baseline = 2
#C Mon Apr 12 17:39:34 2021.  num_events_primary = 1
#C Mon Apr 12 17:39:34 2021.  exit_status = success

#S 35  count(detectors=['temperature'], num=10)
#D Mon Apr 12 17:39:35 2021
#C Mon Apr 12 17:39:35 2021.  plan_type = generator
#C Mon Apr 12 17:39:35 2021.  uid = b6c9ebd4-f468-4ea2-9ca5-58c98dec0038
#MD uid = b6c9ebd4-f468-4ea2-9ca5-58c98dec0038
#MD beamline_id = APS EPICS/Python/Bluesky training VM
#MD detectors = ['temperature']
#MD instrument_name = training
#MD login_id = prjemian@zap
#MD notebook = watch_temperature
#MD num_intervals = 9
#MD num_points = 10
#MD pid = 59652
#MD proposal_id = training-2021-04
#MD versions = {'apstools': '1.4.1', 'bluesky': '1.6.7', 'databroker': '1.2.2', 'epics': '3.4.3', 'h5py': '3.2.1', 'intake': '0.6.2', 'matplotlib': '3.3.4', 'numpy': '1.20.1', 'ophyd': '1.6.1', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}
#P0 
#N 4
#L Epoch_float  Epoch  temperature_setpoint  temperature
0.3207590579986572 0 35.0 34.51313801785306
0.4158611297607422 0 35.0 34.51313801785306
0.4893224239349365 0 35.0 34.51313801785306
0.5678913593292236 1 35.0 34.51313801785306
0.6414732933044434 1 35.0 34.51313801785306
0.715355634689331 1 35.0 34.51313801785306
0.7859370708465576 1 35.0 34.51313801785306
0.8607561588287354 1 35.0 34.51313801785306
0.9332618713378906 1 35.0 34.51313801785306
1.0043187141418457 1 35.0 34.51313801785306
#C Mon Apr 12 17:39:36 2021.  num_events_baseline = 2
#C Mon Apr 12 17:39:36 2021.  num_events_primary = 10
#C Mon Apr 12 17:39:36 2021.  exit_status = success
```

</details>

## User Code File

A common request from instruments is to allow the user to write some python code that can be loaded into the current session.  Since this codedevelopment of this *user code* is often iterative, it must be possible to reload the code without requiring the session to exit and restart.

The local file [`pv_finder.py`](pv_finder.py) provides an example of such a user code file.  Load this file with the command:

    %run -im pv_finder

Note that since the `m` (as module) command option was added, it is not necessary to give the Python extension `.py`.

This file loads the command `findpv(PVNAME)` which identifies the name of the ophyd object(s) using the PV name passed as an argument.  Take for example, we want to find PV `gp:userCalc8.CALC`:

In [12]:
%run -im pv_finder

In [13]:
findpv("gp:userCalc8.CALC")

Subscription value callback exception (EpicsSignal(read_pv='ad:image1:ArrayData', name='adsimdet_image_array_data', parent='adsimdet_image', value=array([], dtype=uint8), timestamp=631152000.0, auto_monitor=True, string=False, write_pv='ad:image1:ArrayData', limits=False, put_complete=False))
Traceback (most recent call last):
  File "/home/prjemian/.local/lib/python3.8/site-packages/ophyd/device.py", line 223, in __get__
    return instance._signals[self.attr]
KeyError: 'shaped_image'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/prjemian/.local/lib/python3.8/site-packages/ophyd/ophydobj.py", line 462, in inner
    cb(*args, **kwargs)
  File "/home/prjemian/.local/lib/python3.8/site-packages/ophyd/signal.py", line 608, in _derived_value_callback
    value = self.inverse(value)
  File "/home/prjemian/.local/lib/python3.8/site-packages/ophyd/areadetector/base.py", line 107, in inverse
    raise RuntimeError(f"Inval

{'read': ['calcs.calc8.calculation', 'temperature.calculation'],
 'write': ['calcs.calc8.calculation', 'temperature.calculation']}

Observe that the same `.calculation` is both readable and writable from two different ophyd objects: `calcs.calc8` and `temperature`.

This command might generate a `KeyError` and/or a `RuntimeError` if the area detector has not passed an image array to the `Image1` plugin.  This error cannot be trapped as it occurs in the underlying subscription.  But, it may be sfaely ignored here.

## databroker

The [`databroker`](https://blueskyproject.io/databroker) package provides 
a Python interface to the database with the experiment data, including 
references to the large file content such as area detector images.  A 
[YAML](https://yaml.org) configuration file connects databroker with a 
specific repository in the MongoDB database server.  In the example here 
([`bluesky_class.yml`](./bluesky_class.yml)), the name of the *catalog* entry
in this file is `class_2021_03`.  It makes two connections to a MongoDB server
running on the same workstation `localhost`.  Both connections are to the 
same MongoDB collection: `class_2021_03-bluesky`.  The name of the file
is not important as long as it is placed in a directory searched by 
`databroker.catalog`.

**Example**

```yml
# file: bluesky_class.yml
# purpose: Configuration file to connect Bluesky databroker with MongoDB
# For 2021-03 Python Training at APS

# Copy to: ~/.local/share/intake/bluesky_class.yml
# Create subdirectories as needed

sources:
  class_2021_03:
    args:
      asset_registry_db: mongodb://localhost:27017/class_2021_03-bluesky
      metadatastore_db: mongodb://localhost:27017/class_2021_03-bluesky
    driver: bluesky-mongo-normalized-catalog
```

When the bluesky session starts, this `class_2021_03` catalog is referenced when
creating the `db` object in the instrument package, in 
`instrument.framework.initialize.py` by these lines:

```python
catalog_name = "class_2021_03"
db = databroker.catalog[catalog_name].v1
logger.info(f"using databroker catalog '{catalog_name}'")
```